# OpenAI Agents SDK Demo

OpenAI have released an [Agents SDK](https://openai.github.io/openai-agents-python/) and [Support MCP](https://openai.github.io/openai-agents-python/mcp/). This demo want to show how to use the SDK to make an application that call multiple agents to do a task.

Install the dependencies library:

In [1]:
%pip install -qU openai-agents==0.0.7 python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Stetup LLM and Agent

Setup OpenAI API url, model and key:

In [2]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

OPENAI_MODEL

'qwen2.5:7b'

Enter your OpenAI API key:  ··········


Setup an agent:

In [3]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI

model= OpenAIChatCompletionsModel(
    model=OPENAI_MODEL,
    openai_client=AsyncOpenAI(base_url=OPENAI_API_BASE,api_key=OPENAI_API_KEY),
)

agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model=model,
)

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-12345***********************cdef. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-12345***********************cdef. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-12345***********************cdef. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key prov

## Running an Agent

In [4]:
result = await Runner.run(
    starting_agent=agent,
    input="tell me a short story",
)
result.final_output

'Once upon a time, in a small village nestled among rolling hills and lush forests, there lived a young girl named Lily. Known for her sparkling green eyes and curly blonde hair, she had a kind heart and an adventurous spirit. Every day after school, as the sun began to dip below the horizon, she would venture into the forest with a picnic basket filled with sandwiches and homemade cookies.\n\nOne afternoon, while wandering through the whispering woods, Lily stumbled upon a small, old shack. The wooden walls were partially covered by creeping vines, and large windows looked directly at her, beckoning her inside. As much as it was forbidden to enter, the temptation was too strong for Lily; she pushed open the creaky door and stepped into the narrow, cozy interior.\n\nInside, the air smelled of old books and forgotten treasures. On one side, rows upon rows of dusty shelves held countless volumes of ancient lore—books nobody had read in decades. In the center of the room, a large round ta

we create a `RunResultStreaming` object by calling `Runner.run_streamed(...)`, we then asynchronously iterate through the streamed events returned by our LLM using the `response.stream_events()` method:

In [5]:
response = Runner.run_streamed(
    starting_agent=agent,
    input="how are you",
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="You're a helpful assistant", handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x10c70fb00>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), type='agent_updated_stream_event')
RawResponsesStreamEvent(data=ResponseCreatedEvent(response=Response(id='__fake_id__', created_at=1743362744.584321, error=None, incomplete_details=None, instructions=None, metadata=None, model='qwen2.5:7b', object='response', output=[], parallel_tool_calls=False, temperature=None, tool_choice='auto', tools=[], top_p=None, max_output_tokens=None, previous_response_id=None, reasoning=

We need to use event.type and event.data to filter out the events we need to progressively display the results returned by LLM.

In [6]:
from openai.types.responses import ResponseTextDeltaEvent

response = Runner.run_streamed(
    starting_agent=agent,
    input="tell me a short story",
)

async for event in response.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Once upon a time, in a vast forest, there lived a kind rabbit named Rosie. Rosie was known throughout the forest for her quick wit and her love for adventure. She would often invite other animals to explore with her.

One sunny morning, she invited a slow-moving tortoise named Theo to join an expedition to find a legendary golden flower hidden deep within the mysterious Whispering Woods. Theo agreed, thinking it would be a pleasant change from his usual meanderings.

They set off at sunrise and wandered through winding paths adorned with sparkling dewdrops. As they explored deeper into the woods, strange whispers seemed to float in the air—sometimes encouraging, sometimes foreboding. 

Late afternoon brought cooler breeze and a bit of mystery when they stumbled upon an old, forgotten garden filled with colorful flowers whispering secrets. Among them was the golden flower, but it was guarded by a tiny fairy named Elara.

Elara told them that to pick the flower meant making a great sacri

## Tools

### Function tools

We can use a `@function_tool` decorator to define a tunction tools:

In [7]:
from agents import function_tool

@function_tool
def add_tool(x: float, y: float) -> float:
    """Add X and Y to get the exact result.
    """
    return x + y

When initializing the `Agent`'s object, we can specify a list of tools via the `tools` parameter:

In [8]:
agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model=model,
    tools=[add_tool]
)

Next we look at the results of the execution:

In [ ]:
response = Runner.run_streamed(
    starting_agent=agent,
    input="What is the result of 9.776 plus 8.625?",
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="You're a helpful assistant", handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x10c70fb00>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[FunctionTool(name='add_tool', description='Add X and Y to get the exact result.', params_json_schema={'properties': {'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}, 'required': ['x', 'y'], 'title': 'add_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b7e4860>, strict_json_schema=True)], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_ch

Unlike before, we see more different values in type, and also different types appear in event.data, so we filter and display them as well:

In [10]:
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
    ResponseTextDeltaEvent,  # text streaming
)

response = Runner.run_streamed(
    starting_agent=agent,
    input="What is the result of 9.776 plus 8.625?",
)

async for event in response.stream_events():
    if event.type == "raw_response_event": 
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            # this is streamed parameters for our tool call
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            # this is streamed final answer tokens
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        # this tells us which agent is currently in use
        print(f"> Current Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        # these are events containing info that we'd typically
        # stream out to a user or some downstream process
        if event.name == "tool_called":
            # this is the collection of our _full_ tool call after our tool
            # tokens have all been streamed
            print()
            print(f"> Tool Called, name: {event.item.raw_item.name}")
            print(f"> Tool Called, args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            # this is the response from our tool execution
            print(f"> Tool Output: {event.item.raw_item['output']}")

> Current Agent: Assistant
{"x":9.776,"y":8.625}
> Tool Called, name: add_tool
> Tool Called, args: {"x":9.776,"y":8.625}
> Tool Output: 18.401
The result of 9.776 plus 8.625 is 18.401.

## Guardrails

You can define an Agent to do guardrails on input and output. 

In [11]:

from pydantic import BaseModel

# define structure of output for any guardrail agents
class GuardrailOutput(BaseModel):
    is_triggered: bool
    reasoning: str

# define an agent that checks if user is asking about political opinions
politics_agent = Agent(
    name="Politics check",
    instructions="Check if the user is asking you about political opinions",
    output_type=GuardrailOutput,
    model=model,
)

Since it's an Agent, we can use it directly: 

In [12]:

query = "What do you think of the Trudeau-led Liberals?"

result = await Runner.run(
    starting_agent=politics_agent, 
    input=query
    )
result.final_output

GuardrailOutput(is_triggered=True, reasoning="The question asks for an opinion on a specific political leader and party, indicating it's seeking a personal viewpoint or sentiment.")

You can use the `@input_guardrail` decorator to define an input guardrail:

In [13]:

from agents import (
    GuardrailFunctionOutput,
    RunContextWrapper,
    input_guardrail
)

# this is the guardrail function that returns GuardrailFunctionOutput object
@input_guardrail
async def politics_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str,
) -> GuardrailFunctionOutput:
    # run agent to check if guardrail is triggered
    response = await Runner.run(starting_agent=politics_agent, input=input)
    # format response into GuardrailFunctionOutput
    return GuardrailFunctionOutput(
        output_info=response.final_output,
        tripwire_triggered=response.final_output.is_triggered,
    )

Now we can initialize our normal agent with the input_guardrails parameter:

In [16]:
agent = Agent(
    name="Assistant",
    instructions=(
        "You're a helpful assistant, remember to always "
        "use the provided tools whenever possible. Do not "
        "rely on your own knowledge too much and instead "
        "use your tools to help you answer queries."
    ),
    model=model,
    tools=[add_tool],
    input_guardrails=[politics_guardrail],  # note this is a list of guardrails
)

Let's try a normal input:

In [ ]:
result = await Runner.run(
    starting_agent=agent,
    input="What is the result of 9.776 plus 8.625?",
)

result.final_output

'The result of 9.776 plus 8.625 is 18.401.'

Let's look at an example of a guarded input:

In [18]:
result = await Runner.run(
    starting_agent=agent,
    input="What are your views on American democracy?",
)

result.final_output

"As an AI, I don't have personal views. However, I can provide information and analyses about American democracy based on available data and academic studies. Would you like me to do that?"